Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [1]:
import Pkg #this line doesn't seem to run either way for me
Pkg.add("Pkg")
Pkg.add("Dates")

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [2]:
"""This activates packages, namely ambulance deployment"""

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`


Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AmbulanceDeployment
└ Loading Random into AmbulanceDeployment from project dependency, future warnings for AmbulanceDeployment are suppressed.

In [3]:
using AmbulanceDeployment

In [4]:
using DataFrames, JLD, Distributions, CSV, Random, JSON, DelimitedFiles, Dates #Plots I don't think we use Plots
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

In [5]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 1 method)

In [22]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)#Distributions.LogNormal(3.089, 0.3) #Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

num_regions = 3200
hospitals = CSV.File(string(local_path,"../test/austin_data_3200/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin_data_3200/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin_data_3200/WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin_data_3200/adjacent_nbhd.csv") |> DataFrame
test_calls = CSV.File("../test/austin_data_3200/austin_test_calls_v3.csv")|> DataFrame 

coverage = CSV.read("../test/austin_data_3200/coverage_regression.csv", DataFrame, header=false) #coverage_real is austin-data
coverage = delete!(coverage,1) #trim df header (1st row)
coverage = select(coverage,2:3234) #trim df header (1st column)
coverage = Matrix(coverage)
#coverage = convert(Array{Bool, 2}, coverage)
coverage = convert(Array{Bool,2},coverage')

# num_regions = 200
# hospitals = CSV.File(string(local_path,"../test/austin_data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin_data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin_data/WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin_data/adjacent_nbhd.csv") |> DataFrame
# test_calls = CSV.File("../test/austin_data/austin_test_calls_v1.csv")|> DataFrame 
# coverage = CSV.read("../test/austin_data/coverage.csv", DataFrame, header=false) #coverage_real is austin-data
# coverage = Matrix(coverage)
# coverage = convert(Array{Bool, 2}, coverage)

solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


include("..//src//problem.jl")

p = DeploymentProblem(
          num_regions,
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )

print("n ambulances ",p.nambulances)
print("\n locations  ",p.nlocations)
print("\n nregions ",p.nregions)
# print("\n train ", p.train)
# print("\n test ", p.test)
print("\n coverage ",size(p.coverage))
print("\n demand ",size(p.demand))
print("\n adjacency ",size(p.adjacency))

#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)
Random.seed!(1234)

40
44
3200
(0, 2)
(0,)
(0,)
(3233, 44)
Array{Bool,2}
(3200, 3200)
BitArray{2}
n ambulances 40
 locations  44
 nregions 3200
 coverage (3233, 44)
 demand (0, 2)
 adjacency (3200, 3200)running 40 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [17]:
test_calls

,interarrival_seconds,neighborhood,dow,Longitude_Of_Emergency,Latitude_Of_Emergency
,Int64,Int64,String,Float64,Float64
1,38,1012,Wed,-97.711,30.26
2,537,220,Wed,-97.695,30.115
3,98,1687,Wed,-97.654,30.379
4,18,1536,Wed,-98.059,30.35
5,475,1621,Wed,-97.716,30.362
6,21,578,Wed,-97.801,30.176
7,263,1169,Wed,-98.064,30.286
8,128,825,Wed,-97.765,30.223
9,62,641,Wed,-97.778,30.193


In [18]:
problem.emergency_calls[:, Symbol("stn$(1)_min")]

1000-element Array{Float64,1}:
  142.25
 1246.6
 1101.43
 2664.6
  925.46
  922.09
 2495.25
  601.92
  668.82
  601.92
  556.96
  407.23
  666.85
    ⋮
 1015.82
 1474.32
  359.04
 1015.82
 1015.82
  925.46
  751.3
  908.34
  953.78
 1385.19
  836.46
  223.54

In [9]:
function logRegression(gridTime)
    gridTime = gridTime / 60
    slope = 0.23868463365149437
    intercept = 1.261777659720721
    ax = log(gridTime)*slope
    new_time = exp(ax + intercept)
    new_time =  new_time * 60
    new_time
end

logRegression (generic function with 1 method)

In [10]:
"""Simulation Engine"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        
        travel_time = ceil(Int, 60*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        println(travel_time)
        travel_time = logRegression(travel_time) #-146 #+ gridTime2Bin(travel_time) #adjust time
        travel_time = ceil(travel_time) #time mod
        println(travel_time)
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    #conveytime = ceil(conveytime * .9) #time mod
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    #returntime = ceil(returntime * .9) #time mod
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [ ]:
"""call simulation"""
#@time df, guiArray = simulate_events!(problem, dispatch)
df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 1012
8535
692.0
calling event id: 2 time: 575 value: 220
38744
993.0
calling event id: 3 time: 673 value: 1687
28574
924.0
calling event id: 4 time: 691 value: 1536
19625
845.0
Ambulance has arrived on the scene for event 1 at time 730
calling event id: 5 time: 1166 value: 1621
22473
872.0
calling event id: 6 time: 1187 value: 578
13295
770.0
calling event id: 7 time: 1450 value: 1169
55619
1083.0
Ambulance has arrived on the scene for event 4 at time 1536
Ambulance has arrived on the scene for event 2 at time 1568
calling event id: 8 time: 1578 value: 825
9157
704.0
Ambulance has arrived on the scene for event 3 at time 1597
calling event id: 9 time: 1640 value: 641
23204
879.0
calling event id: 10 time: 1838 value: 825
9157
704.0
calling event id: 11 time: 1899 value: 765
28704
925.0
calling event id: 12 time: 1901 value: 1070
18159
829.0
Ambulance for event 1 has arrived at the hospital at time 1928
Ambulance has arrived on the scene for event 6 a

54: call from 1561 is a shortfall
calling event id: 55 time: 15714 value: 1437
55: call from 1437 is a shortfall
calling event id: 56 time: 15831 value: 1664
56: call from 1664 is a shortfall
Ambulance for event 10 is returning to the station at 16084
Ambulance for event 8 is returning to the station at 16117
Ambulance from event 10 has returned to station 8
calling event id: 41 time: 16236 value: 888
19067
839.0
This is a shortfallcalling event id: 42 time: 16236 value: 1563
42: call from 1563 is a shortfall
calling event id: 43 time: 16236 value: 578
43: call from 578 is a shortfall
calling event id: 44 time: 16236 value: 1320
44: call from 1320 is a shortfall
calling event id: 45 time: 16236 value: 1669
45: call from 1669 is a shortfall
calling event id: 46 time: 16236 value: 1930
46: call from 1930 is a shortfall
calling event id: 47 time: 16236 value: 772
47: call from 772 is a shortfall
calling event id: 48 time: 16236 value: 1137
48: call from 1137 is a shortfall
calling event i

45: call from 1669 is a shortfall
calling event id: 66 time: 21419 value: 334
66: call from 334 is a shortfall
calling event id: 47 time: 21419 value: 772
47: call from 772 is a shortfall
calling event id: 55 time: 21419 value: 1437
55: call from 1437 is a shortfall
calling event id: 46 time: 21419 value: 1930
46: call from 1930 is a shortfall
calling event id: 59 time: 21419 value: 1438
59: call from 1438 is a shortfall
calling event id: 50 time: 21419 value: 1560
50: call from 1560 is a shortfall
calling event id: 62 time: 21419 value: 1380
62: call from 1380 is a shortfall
calling event id: 43 time: 21419 value: 578
43: call from 578 is a shortfall
calling event id: 65 time: 21419 value: 1501
65: call from 1501 is a shortfall
calling event id: 63 time: 21419 value: 701
63: call from 701 is a shortfall
calling event id: 48 time: 21419 value: 1137
48: call from 1137 is a shortfall
calling event id: 54 time: 21419 value: 1561
54: call from 1561 is a shortfall
calling event id: 58 time:

83: call from 1380 is a shortfall
Ambulance has arrived on the scene for event 66 at time 26948
Ambulance from event 39 has returned to station 32
calling event id: 58 time: 27060 value: 1316
139009
1347.0
This is a shortfallcalling event id: 48 time: 27060 value: 1137
48: call from 1137 is a shortfall
calling event id: 54 time: 27060 value: 1561
54: call from 1561 is a shortfall
calling event id: 46 time: 27060 value: 1930
46: call from 1930 is a shortfall
calling event id: 78 time: 27060 value: 1440
78: call from 1440 is a shortfall
calling event id: 53 time: 27060 value: 824
53: call from 824 is a shortfall
calling event id: 52 time: 27060 value: 1010
52: call from 1010 is a shortfall
calling event id: 60 time: 27060 value: 1625
60: call from 1625 is a shortfall
calling event id: 47 time: 27060 value: 772
47: call from 772 is a shortfall
calling event id: 43 time: 27060 value: 578
43: call from 578 is a shortfall
calling event id: 45 time: 27060 value: 1669
45: call from 1669 is a s

72: call from 1071 is a shortfall
calling event id: 79 time: 28008 value: 702
79: call from 702 is a shortfall
calling event id: 71 time: 28008 value: 766
71: call from 766 is a shortfall
calling event id: 68 time: 28008 value: 642
68: call from 642 is a shortfall
calling event id: 67 time: 28008 value: 823
67: call from 823 is a shortfall
calling event id: 74 time: 28008 value: 1072
74: call from 1072 is a shortfall
calling event id: 51 time: 28008 value: 772
51: call from 772 is a shortfall
calling event id: 70 time: 28008 value: 1440
70: call from 1440 is a shortfall
calling event id: 55 time: 28008 value: 1437
55: call from 1437 is a shortfall
calling event id: 59 time: 28008 value: 1438
59: call from 1438 is a shortfall
calling event id: 84 time: 28008 value: 513
84: call from 513 is a shortfall
calling event id: 85 time: 28008 value: 1010
85: call from 1010 is a shortfall
calling event id: 86 time: 28008 value: 513
86: call from 513 is a shortfall
Ambulance for event 36 is return

...

188: call from 1671 is a shortfall
calling event id: 112 time: 80857 value: 1865
112: call from 1865 is a shortfall
calling event id: 275 time: 80857 value: 825
275: call from 825 is a shortfall
calling event id: 51 time: 80857 value: 772
51: call from 772 is a shortfall
calling event id: 268 time: 80857 value: 1070
268: call from 1070 is a shortfall
calling event id: 173 time: 80857 value: 1612
173: call from 1612 is a shortfall
calling event id: 138 time: 80857 value: 891
138: call from 891 is a shortfall
calling event id: 119 time: 80857 value: 1379
119: call from 1379 is a shortfall
calling event id: 135 time: 80857 value: 455
135: call from 455 is a shortfall
calling event id: 182 time: 80857 value: 2225
182: call from 2225 is a shortfall
calling event id: 270 time: 80857 value: 1420
270: call from 1420 is a shortfall
calling event id: 97 time: 80857 value: 1378
97: call from 1378 is a shortfall
calling event id: 166 time: 80857 value: 768
166: call from 768 is a shortfall
calling

254: call from 1193 is a shortfall
calling event id: 115 time: 80857 value: 1441
115: call from 1441 is a shortfall
calling event id: 246 time: 80857 value: 886
246: call from 886 is a shortfall
calling event id: 194 time: 80857 value: 398
194: call from 398 is a shortfall
calling event id: 242 time: 80857 value: 1989
242: call from 1989 is a shortfall
calling event id: 257 time: 80857 value: 829
257: call from 829 is a shortfall
calling event id: 253 time: 80857 value: 887
253: call from 887 is a shortfall
calling event id: 156 time: 80857 value: 756
156: call from 756 is a shortfall
calling event id: 102 time: 80857 value: 761
102: call from 761 is a shortfall
calling event id: 145 time: 80857 value: 825
145: call from 825 is a shortfall
calling event id: 251 time: 80857 value: 1135
251: call from 1135 is a shortfall
calling event id: 260 time: 80857 value: 831
260: call from 831 is a shortfall
calling event id: 252 time: 80857 value: 1380
252: call from 1380 is a shortfall
calling e

270: call from 1420 is a shortfall
calling event id: 182 time: 82347 value: 2225
182: call from 2225 is a shortfall
calling event id: 83 time: 82347 value: 1380
83: call from 1380 is a shortfall
calling event id: 117 time: 82347 value: 644
117: call from 644 is a shortfall
calling event id: 220 time: 82347 value: 1320
220: call from 1320 is a shortfall
calling event id: 267 time: 82347 value: 702
267: call from 702 is a shortfall
calling event id: 174 time: 82347 value: 1501
174: call from 1501 is a shortfall
calling event id: 231 time: 82347 value: 1194
231: call from 1194 is a shortfall
calling event id: 198 time: 82347 value: 643
198: call from 643 is a shortfall
calling event id: 240 time: 82347 value: 1514
240: call from 1514 is a shortfall
calling event id: 172 time: 82347 value: 1378
172: call from 1378 is a shortfall
calling event id: 100 time: 82347 value: 1131
100: call from 1131 is a shortfall
calling event id: 237 time: 82347 value: 1240
237: call from 1240 is a shortfall
c

Excessive output truncated after 524292 bytes.

calling event id: 219 time: 82347 value: 1255
219: call from 1255 is a shortfall

In [ ]:
println(mean(df[!,"responsetime"]))
println(mean(df[!,"scenetime"]))
println(mean(df[!,"conveytime"]))
println(mean(df[!,"returntime"]))
"""
6.297566666666665
16.250616666666666
24.4176
9.782966666666667
"""

In [10]:
"""batch call simulation"""
# batches = 12
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# using DelimitedFiles

# for i = 1:12
#     """ simulation parameters """
#     p = DeploymentProblem(
#               hourly_calls,
#               adjacent_nbhd,
#               coverage,
#               namb = namb,
#               train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#               )
#     test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#     #update 1: remove negative indices
#     start_ind = (i-1)*1000+1
#     end_ind = (i)*1000
#     println("running $namb ambuances & $start_ind to $end_ind calls")
#     test_calls_batch = test_calls[start_ind:end_ind,:]

#     # distribution of ambulances across all of the stations
#     x = amb_deployment[model_dict[model_name]][namb]

#     #establishes the dispatch problem
#     problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

#     #establishes the dispatch model
#     dispatch = ClosestDispatch(p, problem)

#     #@time df, guiArray = simulate_events!(problem, dispatch)
#     df, guiArray = simulate_events!(problem, dispatch)
#     print(mean(df["responsetime"]))
    
#     filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
#     open(filename, "w") do filename
#         writedlm(filename,  df["responsetime"])
#     end
#     print(filename)
# end

"batch call simulation"

In [11]:
"""all models simulation"""
# turnaround = Distributions.LogNormal(3.65, 0.3)
# lambda = 0
# local_path = ""
# model_name = "Stochastic"
# namb = 40
# ncalls = 1000
# #using AmbulanceDeployment

# pwd()

# cd(currentpath * "/test/")
# pwd()

# #using AmbulanceDeployment
# using Dates

# #include("..//src//dispatch/closestdispatch.jl")
# include("..//src//problem.jl")
# #include("..//src//evaluate.jl") #if you include, you need to include JuMP
# #include("..//src//simulate.jl")

# hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
# coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
# coverage = convert(Array{Bool, 2}, coverage[:, :])
# incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
# solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
# amb_deployment = solverstats["amb_deployment"]
# model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
#           "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


# p = DeploymentProblem(
#           hourly_calls,
#           adjacent_nbhd,
#           coverage,
#           namb = namb,
#           train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#           )
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# test_calls = test_calls[1:ncalls,:]
# println("running $namb ambuances & $ncalls calls")

# # distribution of ambulances across all of the stations
# x = amb_deployment[model_dict[model_name]][namb]

# #establishes the dispatch problem
# problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

# #establishes the dispatch model
# dispatch = ClosestDispatch(p, problem)

# Random.seed!(1234)

"all models simulation"

In [47]:
println(mean(df["responsetime"]))
println(mean(df["scenetime"]))
println(mean(df["conveytime"]))
println(mean(df["returntime"]))

LoadError: [91mArgumentError: syntax df[column] is not supported use df[!, column] instead[39m

println(mean(df["responsetime"]))
println(mean(df["scenetime"]))
println(mean(df["conveytime"]))
println(mean(df["returntime"]))
#### lognromal 3.08
9.813166666666667 <br>
9.134116666666667 <br>
24.4176 <br>
8.908 <br>
#### lognromal 3.65
10.3443 <br>
16.000433333333337 <br>
24.4176 <br>
9.090300000000001 <br>
#### adjust travel time by time_bin weights
10.466350000000002 <br>
16.000433333333334 <br>
24.4176 <br>
9.073 <br>
#### global
7.7477 <br>
16.250616666666666 <br>
24.4176 <br>
9.114916666666668 <br>


In [ ]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

In [60]:
using DelimitedFiles
filename = string("visualization/time_adjust/",model_name,string(namb),"calls",string(ncalls),"stochastic40_3200grid_regression",".csv")
open(filename, "w") do filename
    writedlm(filename,  df[!,"responsetime"])
end
print(filename)

visualization/time_adjust/Stochastic40calls1000stochastic40_210grid_regression.csv

In [ ]:
#using CSV
#filename = string(joshpath,"test/",model_name,string(namb),"calls",string(ncalls),".csv")
#CSV.write(filename, df, delim=",")